# Find the Top 100 White House Visitors in 2011, Using PySpark
A lab in the Hortonworks *HDP Analyst: Data Science* course challenged students to use PySpark to analyze the question of White House visits. The code in this notebook is based on the instructions, but I have made improvements throughout for readabiity and style.

This notebook is intended to run from the HDP Sandbox on port 9999.

In [1]:
from pyspark import SparkContext
sc = SparkContext()

## Load the Data into an RDD
PySpark will likely never support the new DataSet type, so this lab uses the Resilient Distributed Dataset type. PySpark *does* support the DataFrame distributed type, which is processed much more efficiently by the Catalyst processor than RDDs. The operations supported by RDD and DataFrame are disjoint, so you have to know how to use both.

In [2]:
visits = sc.textFile("/user/root/whitehouse/visits.txt")
visits

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

## Mapping
The map function applies a transforming function (i.e., a "map") to each record in the RDD. The Python lambda syntax provides a very fluent way to apply the map to each record.

In [3]:
LASTNAME = 0
FIRSTNAME = 1
VISITEE = 19
def extract_visitors_and_visitee(rec):
    fields = rec.split(',')
    return (", ".join(list([fields[LASTNAME], fields[FIRSTNAME]])), fields[VISITEE])

visitors_and_visitees = visits.map(lambda record: extract_visitors_and_visitee(record))
visitors_and_visitees.take(10)

[(u'ADAMS, CHRISTINE', u'ADAMS'),
 (u'ADAMS, SCHUYLER', u'ADAMS'),
 (u'MAHER, HELEN', u'ADAMS'),
 (u'MAHER, PATRICK', u'ADAMS'),
 (u'MOORE, JESSICA', u'ADAMS'),
 (u'MOORE, LYNN', u'ADAMS'),
 (u'MOORE, RYAN', u'ADAMS'),
 (u'MOORE, THOMAS', u'ADAMS'),
 (u'PARK, ANNE', u'ADAMS'),
 (u'PARK, MAGGIE', u'ADAMS')]

## Filtering
Applying a filter returns an RDD containing only the records that match the filter criteria. In this case, we want an RDD that only contains visits to the President of the United States ("POTUS").

In [4]:
potus_visits = visitors_and_visitees.filter(lambda rec: "POTUS" in rec[1])
potus_visits.take(10)

[(u'BUCKLEY, SUMMER', u'POTUS'),
 (u'CLOONEY, GEORGE', u'POTUS'),
 (u'PRENDERGAST, JOHN', u'POTUS'),
 (u'LANIER, JAZMIN', u'POTUS'),
 (u'MAYNARD, ELIZABETH', u'POTUS'),
 (u'MAYNARD, GREGORY', u'POTUS'),
 (u'MAYNARD, JOANNE', u'POTUS'),
 (u'MAYNARD, KATHERINE', u'POTUS'),
 (u'MAYNARD, PHILIP', u'POTUS'),
 (u'MOHAN, EDWARD', u'POTUS')]

## Reducing to find the count of visits
This is a two-step operation, where the "key" of the operation is the first column (name):
1. Make the value in each record the integer 1.
2. Add the values by key using the `reduceByKey` function. This yields a single record for each key, with a value of the number of visits.

In [5]:
potus_count = potus_visits.map(lambda rec: (rec[0], 1))
potus_count.take(5)

[(u'BUCKLEY, SUMMER', 1),
 (u'CLOONEY, GEORGE', 1),
 (u'PRENDERGAST, JOHN', 1),
 (u'LANIER, JAZMIN', 1),
 (u'MAYNARD, ELIZABETH', 1)]

In [6]:
counts = potus_count.reduceByKey(lambda a, b: a + b)
counts.take(5)

[(u'DONOVAN, MICHAEL', 1),
 (u'PHELPS, DARREN', 3),
 (u'RASSAS, TODD', 1),
 (u'LABOISSIERE, MARTHA', 1),
 (u'SCHUYLER, NICHOLAS', 2)]

## Sorting By Value
It's tricky to sort by value in Spark because it can only sort by key. Therefore you must follow these steps:
1. Reverse each record so that the value becomes the key, and the key becomes the value.
2. Sort the records by key using the aptly named function `sortByKey`.
3. Undo the reversal.
Now we have the original records, sorted by the number of visits to Obama.

In [7]:
counts_reversed = counts.map(lambda rec: (rec[1], rec[0]))
reversed_sorted = counts_reversed.sortByKey(ascending = False)
count_sorted = reversed_sorted.map(lambda rec: (rec[1], rec[0]))
# top 100 visitors to Obama by frequency
count_sorted.take(100)

[(u'PRATHER, ALAN', 16),
 (u'MOTTOLA, ANNAMARIA', 15),
 (u'FRANKE, CHRISTOPHER', 15),
 (u'BOGUSLAW, ROBERT', 14),
 (u'POWERS, CHARLES', 14),
 (u'SATO, FERN', 12),
 (u'FISH, DIANA', 12),
 (u'WALKER, JACKIE', 12),
 (u'WANG, SHENGTSUNG', 12),
 (u'HART, SARAH', 12),
 (u'FETTIG, JASON', 12),
 (u'DEWEY, GLENN', 11),
 (u'WILSON, PETER', 11),
 (u'BAILEY, JANET', 11),
 (u'BOTELHO, MARCIO', 11),
 (u'WILLINGHAM, DONNA', 11),
 (u'SMITH, CHAERIM', 10),
 (u'CHUDACOFF, CLAUDIA', 10),
 (u'JARRETT, VALERIE', 10),
 (u'COLBURN, MICHAEL', 10),
 (u'SABO, ERIC', 10),
 (u'AXELROD, DAVID', 10),
 (u'MADRID, REGINO', 10),
 (u'TRUMKA, RICHARD', 9),
 (u'DALEY, WILLIAM', 9),
 (u'TRAN, TAM', 9),
 (u'RAKERS, MICHELLE', 9),
 (u'SEARFOSS, STEVEN', 9),
 (u'SHIEH, FRANCIS', 9),
 (u'HOLDER, ERIC', 9),
 (u'MERGEN, MICHAEL', 8),
 (u'PELOSI, NANCY', 8),
 (u'CLAY, AARON', 8),
 (u'WILSON, RUSSELL', 8),
 (u'SHER, SUSAN', 8),
 (u'COLLINS, MICHAEL', 8),
 (u'ZENOBIA, HEATHER', 8),
 (u'RIDLINGTON, GREGORY', 7),
 (u'LUBIN, MATTHEW'

## Results
Now we know who some of the most important Presidential advisors (e.g., Valerie Jarrett), members of Congress (e.g., Nancy Pelosi), and public figures (e.g., Governor John Hickenlooper) were in 2011, as measured by their access to President Obama.